In [ ]:
###### pretraining the model

from datasets import load_from_disk

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

from datasets import load_dataset
import glob
import tokenizers
from transformers import Trainer, TrainingArguments, LineByLineTextDataset, BertModel
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling
from transformers import AutoTokenizer

dataset = load_from_disk("tokenized_dataset/")
# dataset = load_dataset("tokenized_dataset/")

# max_seq_length = 128
tokenizer = AutoTokenizer.from_pretrained('BERTLLM/')
config = BertConfig( vocab_size = 75000, 
                    hidden_size = 768, 
                    num_hidden_layers = 12,
                    num_attention_heads = 12,
                    max_position_embeddings = 512)

model = BertForMaskedLM(config)
display(model.num_parameters())


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True,
                                               mlm_probability=0.15)
epochs = 15
save_steps = 20_000 #save checkpoint every 10000 steps
batch_size = 256

training_args = TrainingArguments(
    output_dir = 'BERTLLM/',
    overwrite_output_dir=True,
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    save_steps = save_steps,
    save_total_limit = 3, #only save the last 5 checkpoints
    fp16=True,
    # tf32 = True,
    learning_rate = 5e-5,  # 5e-5 is the default
    logging_steps = 10_000,
    gradient_accumulation_steps=2,
    logging_first_step = True,
    # gradient_checkpointing=True,
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset=dataset

)


# trainer.train(resume_from_checkpoint=True)
trainer.train()
trainer.save_model('BERTLLM/')

Loading dataset from disk:   0%|          | 0/242 [00:00<?, ?it/s]

143717880

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
1,11.318000
10000,6.055400
20000,4.797900
30000,4.346800
40000,4.082000
50000,3.907800
60000,3.775700
70000,3.676000
80000,3.593800
90000,3.523300


In [ ]:
loss = 5e-5

10117/2585790 1:09:14 < 293:51:04, 2.43 it/s, Epoch 0.04/10]

Step 	Training Loss
1 	11.351700
10000 	6.048200